# Introduction

In this notebook we present a set of basic tests of correctness of auxiliary
classes and methods for conducting statistical inference based on ERGM
null models implemented in `pathcensus` package. Everything we show here
is also tested in the automated unit test suite, but we provide also
a notebook-based confirmation as it is arguably much easier to follow.

We focus on testing methods for estimating $p$-values of observed
edge/node/graph structural coefficients relative to a null model
(see notebook `2-null-models-test.ipynb` for more details).
More conretely, we will consider the two following test cases:

1. **Erdős–Rényi random graph.** In this case we expect all structural
   coefficients on all levels (edges/nodes/graph) to be insignificant
   with type I error rate not greater than $\alpha$ when using a proper
   adjustment for multiple testing (FDR procedure by Benjamini and Hochberg).
2. **Random geometric graph (RGG),** As above but in this case we expect
   similarity coefficients to be significantly larger than null model
   expectations.

For null model we will use Undirected Binary Configuration Model (UBCM)
implemented in `pathcensus` package.

In [1]:
import numpy as np
import igraph as ig
from pathcensus import PathCensus
from pathcensus.nullmodels import UBCM
from pathcensus.inference import Inference
from pathcensus.utils import set_seed

def make_er_graph(n, dbar):
    p = dbar / (n-1)
    return ig.Graph.Erdos_Renyi(n, p=p, directed=False)

def make_rgg(n, dbar):
    radius = np.sqrt(dbar/(np.pi*(n-1)))
    return ig.Graph.GRG(n, radius=radius, torus=True)

# Global parameters
# -----------------
N_NODES   = 100     # number of nodes in random graphs
KBAR      = 10      # expected average degree in random graphs
ALPHA     = 0.01    # Upper bound for type I error rate with the FDR correction
N_SAMPLES = 100     # number of samples used for estimating p-values 

## Implementing statistical inference procedure

In this project we conduct most of statistical inference based on simple
exponential random graph models (ERGM), primarily different variants
of the configuration model.

Comparisons between observed values of various graph properties and
expectations based on a null model are done according to the following scheme:

1. Calculate observed values of graph statistics of interest and
   index each node with its corresponding sufficient statistic(s).
   Note that in the models we use sufficient statistics are always defined 
   for nodes (i.e. degree sequence in the standard configuration model).
2. Sample $R$ randomized realization from a null model of choice.
3. Index nodes in each randomized graph with values of their corresponding
   sufficient statistics.
4. Calculate graph statistics on $R$ randomized graph.
5. Group simulated data by unique values of sufficient statistics.
   In the case of nodes these are original sufficient statistics
   (always defined for nodes in our case) and in the case of edges
   these are unique combinations of sufficient statistics
   (possibly coarse grained to avoid having too sparse data).
   In the case of graph-level statistics no grouping is necessary.
6. Compare observed values against the simulated values grouped
   as described above where values for individual nodes/edges
   are compared against distributions corresponding to their
   values of sufficient statistics.

**NOTE.** In this approach only one-sided tests are really possible.
          Moreover, estimating p-values for edge-wise statistics
          is currently problematic due to the need of coarse-graining
          (and it is not yet clear what is the optimal strategy).

In practice we usually implement the above procedure using a helper
`Inference` class defined in `pathcensus` package which abstracts away
most of tedious programming logic and requires the end user to implement
only `statistics` method defining the actual graph statistics we want
to calculate.

Below we do this for the sake of the test of $p$-values estimation methods.

In [2]:
def statistics(graph, method, *args, **kwds):
    """Function computing graph statistics for which inference
    is to be run.
    
    Parameters
    ----------
    graph
        A graph-like object.
    method
        Name of the method (string) defined on 
        :py:class:`pathcensus.PathCensus`.
    
    Returns
    -------
    data
        Data frame or series with grah statistics.
    """
    paths = PathCensus(graph)
    method = getattr(paths, method)
    return method(*args, **kwds) 

## ER random graph

In this case we expect all coefficients to be insignificant.
We use one-sided tests checking with "greater" alternative hypothesis.

In [3]:
# This sets seed of random, numpy and numba
set_seed(371)

graph = make_er_graph(N_NODES, KBAR)
ubcm  = UBCM(graph)
ubcm.fit()

infer    = Inference(graph, ubcm, statistics)
null_kws = dict(progress=True)

### Node-wise coefficients

In [4]:
# ONE-TAILED TEST FOR NODE-WISE STRUCTURAL COEFFICIENTS
data, null = infer.init_comparison(
    n=N_SAMPLES,
    method="coefs",
    mode="nodes",
    null_kws=null_kws
)
pvals = infer.estimate_pvalues(data, null, alternative="greater")

# CHECK IF THE FRACTION OF SIGNIFICANT VALUES
# DOES NOT EXCEED ALPHA
pvals_frac = (pvals.values <= ALPHA).mean()
pvals_frac, pvals_frac <= ALPHA

100%|██████████| 100/100 [00:04<00:00, 22.33it/s]


(0.0, True)

### Global coefficients

In [5]:
# ONE-TAILED TEST FOR GLOBAL STRUCTURAL COEFFICIENTS
data, null = infer.init_comparison(
    n=N_SAMPLES,
    method="coefs",
    mode="global",
    null_kws=null_kws
)
pvals = infer.estimate_pvalues(data, null, alternative="greater")

# CHECK IF THE FRACTION OF SIGNIFICANT VALUES
# DOES NOT EXCEED ALPHA
pvals_frac = (pvals.values <= ALPHA).mean()
pvals_frac, pvals_frac <= ALPHA

100%|██████████| 100/100 [00:04<00:00, 20.76it/s]


(0.0, True)

## Random geometric graph

In this case we expect significant presence of similarity
and not significant results for complementarity.

In [6]:
set_seed(7171)

graph = make_rgg(N_NODES, KBAR)
ubcm  = UBCM(graph)
ubcm.fit()

infer = Inference(graph, ubcm, statistics)

### Node-wise similarity

In this case we expect a fraction of significant results greater than $\alpha$
using a one-sided test.

In [7]:
# ONE-TAILED TEST FOR NODE-WISE SIMILARITY COEFFICIENTS
# (W0-COMPLEMENTARITY IS USED)
data, null = infer.init_comparison(
    n=N_SAMPLES,
    method="simcoefs",
    mode="nodes",
    null_kws=null_kws
)
pvals = infer.estimate_pvalues(data, null, alternative="greater")

# CHECK IF THE FRACTION OF SIGNIFICANT VALUES
# EXCEED ALPHA
pvals_frac = (pvals.values <= ALPHA).mean()
pvals_frac, pvals_frac > ALPHA

100%|██████████| 100/100 [00:03<00:00, 32.34it/s]


(0.9833333333333333, True)

### Global similarity

In [8]:
# ONE-TAILED TEST FOR NODE-WISE SIMILARITY COEFFICIENTS
# (W0-COMPLEMENTARITY IS USED)
data, null = infer.init_comparison(
    n=N_SAMPLES,
    method="simcoefs",
    mode="global",
    null_kws=null_kws
)
pvals = infer.estimate_pvalues(data, null, alternative="greater")

# CHECK IF THE FRACTION OF SIGNIFICANT VALUES
# EXCEED ALPHA
pvals_frac = (pvals.values <= ALPHA).mean()
pvals_frac, pvals_frac > ALPHA

100%|██████████| 100/100 [00:02<00:00, 34.64it/s]


(1.0, True)

### Node-wise complementarity

We expect no significant results.

In [9]:
# ONE-TAILED TEST FOR EDGE-WISE SIMILARITY COEFFICIENTS
# (W0-COMPLEMENTARITY IS USED)
data, null = infer.init_comparison(
    n=N_SAMPLES,
    method="compcoefs",
    mode="nodes",
    null_kws=null_kws
)
pvals = infer.estimate_pvalues(data, null, alternative="greater")

# CHECK IF THE FRACTION OF SIGNIFICANT VALUES
# EXCEED ALPHA
pvals_frac = (pvals.values <= ALPHA).mean()
pvals_frac, pvals_frac <= ALPHA

100%|██████████| 100/100 [00:02<00:00, 39.55it/s]


(0.0, True)

### Global complementarity

In [10]:
# ONE-TAILED TEST FOR EDGE-WISE SIMILARITY COEFFICIENTS
# (W0-COMPLEMENTARITY IS USED)
data, null = infer.init_comparison(
    n=N_SAMPLES,
    method="compcoefs",
    mode="global",
    null_kws=null_kws
)
pvals = infer.estimate_pvalues(data, null, alternative="greater")

# CHECK IF THE FRACTION OF SIGNIFICANT VALUES
# EXCEED ALPHA
pvals_frac = (pvals.values <= ALPHA).mean()
pvals_frac, pvals_frac <= ALPHA

100%|██████████| 100/100 [00:03<00:00, 31.10it/s]


(0.0, True)